# ADT SOAMC SQS WPS-T INTIGRATION

In [ ]:
%%bash
pip install requests
pip install typer
pip install multiprocessing_logging

## Config file

**Config file located in /home/jovyan/soamc-python-sqs-client/sqsconfig.py**

**Update the values as needed**


[AWS_SQS_QUEUE]

AWS_ACCOUNT_ID={{ AWS_ACCOUNT_ID }}

region_name=us-west-2

aws_access_key={{ AWS_ACCESS_KEY }}

aws_secret_key={{ AWS_SECRET_KEY }}

listener_queue = {{ SQS_LISTENER_QUEUE }}

poll_interval = 60

queue_visibility_timeout = 600

error_queue_name = None

error_queue_visibility_timeout = 600

force_delete = False

wait_time = 0

max_number_of_messages =  1

queue_url = {{ QUEUE_URL }}


[DAEMON]

PID_FILE_PATH={{ PID_FILE_PATH }}/daemon_pid.pid

DAEMON_OUTPUT_OVERWRITE=False 

DAEMON_OUTPUT_FILE={{ DAEMON_OUTPUT_FILE_PATH }}/listener_out.log

DAEMON_ERROR_FILE={{ DAEMON_OUTPUT_FILE_PATH }}/listener_error.log

DAEMON_STDIN=/dev/null


[ADES_WPS-T_SERVER]

wps_server_url=http://127.0.0.1:500


In [1]:
CONFIG_FILER_PATH = r'/home/jovyan/soamc-python-sqs-client/sqsconfig.py'

## Utility Functions

In [ ]:
import logging 
import typer
import logging
import configparser
import json
import os
from sqs_client.factories import ReplyQueueFactory, PublisherFactory
from sqs_client.message import RequestMessage
from sqs_client.exceptions import ReplyTimeout

config = configparser.ConfigParser()
config.read(CONFIG_FILER_PATH)
print(config.sections())

os.environ["AWS_ACCOUNT_ID"] = config["AWS_SQS_QUEUE"]["AWS_ACCOUNT_ID"]
os.environ["AWS_ACCESS_KEY"] = config["AWS_SQS_QUEUE"]["aws_access_key"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["AWS_SQS_QUEUE"]["aws_secret_key"]
print(os.environ["AWS_ACCOUNT_ID"])
wps_server = config["ADES_WPS-T_SERVER"]["wps_server_url"]
queue_url = config["AWS_SQS_QUEUE"]['queue_url']

reply_queue = ReplyQueueFactory(
    name='reply_queue_',
    access_key=config["AWS_SQS_QUEUE"]["aws_access_key"],
    secret_key=config["AWS_SQS_QUEUE"]["aws_secret_key"],
    region_name=config["AWS_SQS_QUEUE"]['region_name']
).build()

publisher = PublisherFactory(
    access_key=config["AWS_SQS_QUEUE"]["aws_access_key"],
    secret_key=config["AWS_SQS_QUEUE"]["aws_secret_key"],
    region_name=config["AWS_SQS_QUEUE"]['region_name']
).build()

class soamc_submitter():
    def __init__(self):
       pass

    def submit_message(self, data):
        message = RequestMessage(
            body= json.dumps(data),
            queue_url= queue_url,
            reply_queue=reply_queue
        )
        publisher.send_message(message)

        try:
            response = message.get_response(timeout=20)
            #print(response.body)
            return json.loads(response.body)
        except ReplyTimeout:
            return {"Error:": "Timeout"}
        except Exception as e:
            return {"Error": str(e)}
        finally:
            reply_queue.remove_queue()


    def info(self):
        data = {'job_type': 'getLandingPage'}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def get_processes(self):
        data = {'job_type': 'getProcesses'}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def deployProcess(self):
        data = {'job_type': 'deployProcess'}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def process_desc(self, process_id: str):
        print(process_id)
        data = {'job_type': 'getProcessDescription', 'process_id' : process_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def delete_process(self, process_id: str):
        print(process_id)
        data = {'job_type': 'undeployProcess', 'process_id' : process_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def job_list(self, process_id: str):
        data = {'job_type': 'getJobList', 'process_id' : process_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def run_process(self, process_id: str):
        print(self, process_id)
        data = {'job_type': 'execute', 'process_id' : process_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def job_status(self, process_id: str, job_id:str):
        print(process_id)
        data = {'job_type': 'getStatus', 'process_id' : process_id, 'job_id': job_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def delete_job(self, process_id: str, job_id:str):
        print(process_id)
        data = {'job_type': 'dismiss', 'process_id' : process_id, 'job_id': job_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


    def job_result(self, process_id: str, job_id:str):
        print(process_id)
        data = {'job_type': 'getResult', 'process_id' : process_id, 'job_id': job_id}
        response = self.submit_message(data)
        print(json.dumps(response, indent=2))


        
soamc_submitter = soamc_submitter()
        

## All the Available End Points

In [ ]:
info = soamc_submitter.info()
print(info)

## Get All the Processs

In [ ]:
processes = soamc_submitter.get_processes()
print(processes)

## Description of a Particular Process

In [ ]:
process_desc = soamc_submitter.process_desc("proc1")
print(process_desc)

## List all Jobs of a Particular Process

In [ ]:
job_list = soamc_submitter.job_list("proc1")
print(job_list)

## Result of a Particular Process

In [ ]:
result = soamc_submitter.job_result("proc1", "job1")
print(result)


## Status of a Job

In [ ]:
status = soamc_submitter.job_status("proc1", "job1")
print(status)

## Delete a job

In [ ]:
response = soamc_submitter.delete_job("proc1", "job1")
print(response)

## Delete a Process

In [ ]:
response = soamc_submitter.delete_process("proc1")
print(response)

## Deploy all The Processes

In [ ]:
response = soamc_submitter.deployProcess()
print(response)

## Execute a Process

In [ ]:
response = soamc_submitter.run_process("proc1")
print(response)